In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
! pip install transformers
! pip install flask
! pip install flask-ngrok
! pip install -U flask-cors

In [ ]:
modelNames = ["BERTModel-Twitter", "BERTModel-IMDB", "BERTModel-Emotions"]
labels = [['Positive','Negative','Neutral'], ['Negative','Positive'], ['Sadness', 'Anger', 'Love', 'Surprise', 'Fear', 'Happiness']]
DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [ ]:
from flask import Flask, jsonify, request, Response
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd
import numpy as np
import json

# Function that loads a saved model with its test and validation sets
def loadModel(model_save_name):
    path = F"/content/gdrive/MyDrive/Colab Notebooks/ml-blockchain/savedModels/{model_save_name}" 
    testSetPath = F"/content/gdrive/MyDrive/Colab Notebooks/ml-blockchain/savedModels/{model_save_name}-test.csv"
    validationSetPath = F"/content/gdrive/MyDrive/Colab Notebooks/ml-blockchain/savedModels/{model_save_name}-validation.csv"

    test = pd.read_csv(testSetPath)
    validation = pd.read_csv(validationSetPath)
    test = test.iloc[: , 1:]
    validation = validation.iloc[: , 1:]

    loaded_model = TFBertForSequenceClassification.from_pretrained(path, local_files_only=True)
    return loaded_model,tokenizer,test,validation


# Functions that preprocess input data before training using tokenizer
def convert_data_to_examples_single(inputDataset, DATA_COLUMN, LABEL_COLUMN): 
    train_InputExamples = inputDataset.apply(lambda x: InputExample(guid=None,
                                                        text_a = x[DATA_COLUMN], 
                                                        text_b = None,
                                                        label = x[LABEL_COLUMN]), axis = 1)  
    return train_InputExamples
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []
    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, 
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, 
            truncation=True
        )
        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])
        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )
    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )
    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

# Function that retrains and replaces a loaded model using a new train set 
def trainModel(loadedModel,tokenizer,validation,train,model_save_name):
    # pass validation set through tokenizer
    validation_InputExamples = convert_data_to_examples_single(validation, DATA_COLUMN, LABEL_COLUMN)
    validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
    validation_data = validation_data.batch(32)
    # pass train set through tokenizer
    train_InputExamples = convert_data_to_examples_single(train, DATA_COLUMN, LABEL_COLUMN)
    train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
    train_data = train_data.batch(32).repeat(2)
    # Retrain model
    loadedModel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
    loadedModel.fit(train_data, epochs=2, validation_data=validation_data)
    # Replace model
    path = F"/content/gdrive/MyDrive/Colab Notebooks/ml-blockchain/savedModels/{model_save_name}" 
    loadedModel.save_pretrained(path)
    return loadedModel

# Function that evaluates a model using its test set
def evaluateModel(loadedModel,tokenizer,test):
    # pass test set through tokenizer
    test_inputExamples = convert_data_to_examples_single(test, DATA_COLUMN, LABEL_COLUMN)
    test_data = convert_examples_to_tf_dataset(list(test_inputExamples), tokenizer)
    test_data = test_data.batch(32)
    # Evaluate model
    loadedModel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
    evaluation = loadedModel.evaluate(test_data)
    return (evaluation)

# Fucntion that predicts a laber for a caption using a model
def testModel(loadedModel,tokenizer,pred_sentences,labels):
    # pass caption through tokenizer
    tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
    # predict caption
    tf_outputs = loadedModel(tf_batch)
    tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
    label = tf.argmax(tf_predictions, axis=1)
    label = label.numpy()
    return labels[label[0]]

###################################################################################################################

app = Flask(__name__)
run_with_ngrok(app) # expose port in public URL
CORS(app) # Disable CORS restrictions

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # Download tokenizer
# Create empty arrays to save models, test sets and validation sets
loadedModel = [None] * len(modelNames)
test = [None] * len(modelNames)
validation = [None] * len(modelNames)

# Load all saved models
for id,modelName in enumerate(modelNames):
  print(id,modelName)
  loadedModel[id],tokenizer,test[id],validation[id] = loadModel(modelName)

# /train endpoint: Retrain model
@app.route('/train', methods=['POST'])
def trainFun():
    print('Starting Training')
    modelId = int(request.args.get('model'))

    # trainSetInput = request.json # used if request contains json file
    trainSetInput = json.loads(request.data.decode('utf-8')) # used if request contains raw text

    trainDF = pd.DataFrame(trainSetInput)
    del trainDF['id']
    del trainDF['proposedLbl']
    del trainDF['goodData']
    trainDF.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
    trainModel(loadedModel[modelId],tokenizer,validation[modelId],trainDF,modelNames[modelId])
    
    # response = Response(status=200)
    response = jsonify(status="ok")
    return response


# /evaluate endpoint: Evaluate model
@app.route('/evaluate')
def evaluateFun():
    print('Starting Evaluation')
    modelId = int(request.args.get('model'))

    eval = evaluateModel(loadedModel[modelId],tokenizer,test[modelId])
    [loss,acc] = eval

    response = jsonify(loss=loss,acc=acc)
    return response


# /test endpoint: Predicts label for a caption
@app.route('/test', methods=['POST'])
def testFun():
    modelId = int(request.args.get('model'))

    dataIn = json.loads(request.data.decode('utf-8'))
    pred_sent = dataIn['caption'] # used if request contains raw text
    # pred_sent = request.json['caption'] # used if request contains json file

    pred = testModel(loadedModel[modelId],tokenizer,pred_sent,labels[modelId])

    response = jsonify(prediction=pred)
    return response


if (__name__ == "__main__"):
    print('Server Running🚀')
    app.run()



